# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [42]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'K/AS AAGRLS(SLT MNLT)QOAEGE DBCEN', 'calculationPrice': 'tops', 'open': None, 'openTime': None, 'openSource': 'aoffclii', 'close': None, 'closeTime': None, 'closeSource': 'ifolfiac', 'high': None, 'highTime': None, 'highSource': None, 'low': None, 'lowTime': None, 'lowSource': None, 'latestPrice': 131.44, 'latestSource': 'IEX real time price', 'latestTime': '3:35:46 PM', 'latestUpdate': 1661053501667, 'latestVolume': None, 'iexRealtimePrice': 127.85, 'iexRealtimeSize': 105, 'iexLastUpdated': 1685332650013, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 135.64, 'previousVolume': 87722596, 'change': -3.44, 'changePercent': -0.02705, 'volume': None, 'iexMarketPercent': 0.0190850401904183, 'iexVolume': 1559498, 'avgTotalVolume': 98689769, 'iexB

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [43]:
price = data['latestPrice']
pe_ratio = data['peRatio']

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [44]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [45]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A'
            ],
            index = my_columns
        ),
            ignore_index = True
        )
        

## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [46]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis = 1, inplace=True)

In [47]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,38.208,2.3,N/A
1,AIV,5.017,4.87,N/A
2,BIO,625.700,4.9,N/A
3,UNM,27.638,6.98,N/A
4,ALL,109.129,7.61,N/A
5,AFL,49.267,7.97,N/A
6,CPB,47.700,8.03,N/A
7,EBAY,64.290,8.17,N/A
8,CE,137.790,8.4,N/A
9,PHM,47.500,9.11,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [48]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [50]:
portfolio_input()

Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [55]:
position_size = float(portfolio_size)/len(final_dataframe.index)

for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,38.208,2.3,523
1,AIV,5.017,4.87,3986
2,BIO,625.700,4.9,31
3,UNM,27.638,6.98,723
4,ALL,109.129,7.61,183
5,AFL,49.267,7.97,405
6,CPB,47.700,8.03,419
7,EBAY,64.290,8.17,311
8,CE,137.790,8.4,145
9,PHM,47.500,9.11,421


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [70]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

#Price-to-earnings ratio
pe_ration = data[symbol]['quote']['peRatio']

#Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

#Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

#Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit
print(ev_to_gross_profit)

19.380534907443348


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [83]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}' 
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
            
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
        
        rv_dataframe = rv_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            'N/A',
            data[symbol]['quote']['peRatio'],
            'N/A',
            data[symbol]['advanced-stats']['priceToBook'],
            'N/A',
            data[symbol]['advanced-stats']['priceToSales'],
            'N/A',
            ev_to_ebitda,
            'N/A',
            ev_to_gross_profit,
            'N/A',
            'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

In [84]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,127.20,N/A,54.3,N/A,8.15,N/A,7.54,N/A,34.157662,N/A,13.733456,N/A,N/A
1,AAL,21.36,N/A,-1.16,N/A,-1.75,N/A,0.6913,N/A,-4.237737,N/A,2.178075,N/A,N/A
2,AAP,168.67,N/A,25.0,N/A,3.02,N/A,1.15,N/A,11.473334,N/A,2.576573,N/A,N/A
3,AAPL,126.43,N/A,34.95,N/A,33.22,N/A,7.49,N/A,25.774387,N/A,19.621210,N/A,N/A
4,ABBV,108.36,N/A,24.47,N/A,12.71,N/A,4.72,N/A,16.259611,N/A,9.448072,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.17,N/A,32.04,N/A,-4,N/A,5.89,N/A,23.405485,N/A,9.620980,N/A,N/A
501,ZBH,166.97,N/A,-220.32,N/A,2.76,N/A,4.8,N/A,27.105492,N/A,7.973411,N/A,N/A
502,ZBRA,490.35,N/A,53.94,N/A,12.72,N/A,6,N/A,35.838846,N/A,13.586335,N/A,N/A
503,ZION,53.53,N/A,23.3,N/A,1.24,N/A,2.9,N/A,12.366926,N/A,3.305635,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [87]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,239.880,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,245.360,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.84,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,47.304,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,51.350,N/A,14.87,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,54.540,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,33.680,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,176.900,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2140.560,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
326,MYL,15.880,N/A,30.92,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [92]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [93]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [148]:
from scipy.stats import percentileofscore as score

metrics = {
    'Price-to-Earnings Ratio': 'PE Percentile',
    'Price-to-Book Ratio': 'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric]) / 100
        
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,27.610,1810,7.10,0.52,0.5000,0.24,0.4067,0.2,3.447339,0.16,0.396879,0.04,5.861386
1,AIG,45.470,1099,-8.07,0.32,0.6061,0.28,0.8266,0.48,5.042930,0.22,0.827923,0.16,6.217822
2,MCK,181.350,275,-7.20,0.36,-60.8400,0.06,0.1200,0.04,6.876861,0.4,2.648461,0.62,6.257426
3,FTI,7.380,6775,-0.57,0.48,0.8119,0.36,0.2533,0.08,2.085738,0.12,1.363498,0.32,6.415842
4,MET,59.680,837,9.79,0.62,0.7020,0.32,0.7627,0.46,5.127948,0.24,0.754897,0.14,7.683168
5,AAL,21.360,2340,-1.16,0.44,-1.7500,0.2,0.6913,0.4,-4.237737,0.08,2.178075,0.54,7.881188
6,ALL,108.560,460,7.72,0.54,1.2400,0.68,0.7075,0.42,2.318074,0.14,0.715466,0.12,8.356436
7,ABC,109.810,455,-6.75,0.38,-31.3100,0.08,0.1159,0.02,7.646935,0.6,3.661924,0.78,8.435644
8,DXC,26.550,1883,-2.40,0.4,1.1900,0.57,0.3730,0.16,5.132046,0.26,2.318507,0.58,8.574257
9,F,11.900,4201,-37.77,0.14,1.4700,0.76,0.3572,0.14,5.953324,0.3,2.771562,0.64,8.633663


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [149]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,27.610,1810,7.10,0.52,0.5000,0.24,0.4067,0.2,3.447339,0.16,0.396879,0.04,0.232
1,AIG,45.470,1099,-8.07,0.32,0.6061,0.28,0.8266,0.48,5.042930,0.22,0.827923,0.16,0.292
2,MCK,181.350,275,-7.20,0.36,-60.8400,0.06,0.1200,0.04,6.876861,0.4,2.648461,0.62,0.296
3,FTI,7.380,6775,-0.57,0.48,0.8119,0.36,0.2533,0.08,2.085738,0.12,1.363498,0.32,0.272
4,MET,59.680,837,9.79,0.62,0.7020,0.32,0.7627,0.46,5.127948,0.24,0.754897,0.14,0.356
5,AAL,21.360,2340,-1.16,0.44,-1.7500,0.2,0.6913,0.4,-4.237737,0.08,2.178075,0.54,0.332
6,ALL,108.560,460,7.72,0.54,1.2400,0.68,0.7075,0.42,2.318074,0.14,0.715466,0.12,0.38
7,ABC,109.810,455,-6.75,0.38,-31.3100,0.08,0.1159,0.02,7.646935,0.6,3.661924,0.78,0.372
8,DXC,26.550,1883,-2.40,0.4,1.1900,0.57,0.3730,0.16,5.132046,0.26,2.318507,0.58,0.394
9,F,11.900,4201,-37.77,0.14,1.4700,0.76,0.3572,0.14,5.953324,0.3,2.771562,0.64,0.396


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [150]:
rv_dataframe.sort_values('RV Score', ascending = True, inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,27.610,1810,7.10,0.52,0.5000,0.24,0.4067,0.2,3.447339,0.16,0.396879,0.04,0.232
1,FTI,7.380,6775,-0.57,0.48,0.8119,0.36,0.2533,0.08,2.085738,0.12,1.363498,0.32,0.272
2,BA,217.730,229,-10.70,0.26,-7.0100,0.16,2.2800,0.96,-15.572623,0.04,-29.337555,0.02,0.288
3,AIG,45.470,1099,-8.07,0.32,0.6061,0.28,0.8266,0.48,5.042930,0.22,0.827923,0.16,0.292
4,MCK,181.350,275,-7.20,0.36,-60.8400,0.06,0.1200,0.04,6.876861,0.4,2.648461,0.62,0.296
5,AAL,21.360,2340,-1.16,0.44,-1.7500,0.2,0.6913,0.4,-4.237737,0.08,2.178075,0.54,0.332
6,PRU,88.960,562,-334.01,0.06,0.5070,0.26,0.5531,0.28,24.191095,1.0,0.566349,0.08,0.336
7,MET,59.680,837,9.79,0.62,0.7020,0.32,0.7627,0.46,5.127948,0.24,0.754897,0.14,0.356
8,LYV,90.290,553,-13.10,0.24,-201.5100,0.02,4.2800,1.0,-24.909929,0.02,2.259892,0.56,0.368
9,ABC,109.810,455,-6.75,0.38,-31.3100,0.08,0.1159,0.02,7.646935,0.6,3.661924,0.78,0.372


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [151]:
portfolio_input()

Enter the value of your portfolio:2500000


In [152]:
position_size = float(portfolio_size)/len(rv_dataframe.index)
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row, 'Price'])
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,27.610,1810,7.10,0.52,0.5000,0.24,0.4067,0.2,3.447339,0.16,0.396879,0.04,0.232
1,FTI,7.380,6775,-0.57,0.48,0.8119,0.36,0.2533,0.08,2.085738,0.12,1.363498,0.32,0.272
2,BA,217.730,229,-10.70,0.26,-7.0100,0.16,2.2800,0.96,-15.572623,0.04,-29.337555,0.02,0.288
3,AIG,45.470,1099,-8.07,0.32,0.6061,0.28,0.8266,0.48,5.042930,0.22,0.827923,0.16,0.292
4,MCK,181.350,275,-7.20,0.36,-60.8400,0.06,0.1200,0.04,6.876861,0.4,2.648461,0.62,0.296
5,AAL,21.360,2340,-1.16,0.44,-1.7500,0.2,0.6913,0.4,-4.237737,0.08,2.178075,0.54,0.332
6,PRU,88.960,562,-334.01,0.06,0.5070,0.26,0.5531,0.28,24.191095,1.0,0.566349,0.08,0.336
7,MET,59.680,837,9.79,0.62,0.7020,0.32,0.7627,0.46,5.127948,0.24,0.754897,0.14,0.356
8,LYV,90.290,553,-13.10,0.24,-201.5100,0.02,4.2800,1.0,-24.909929,0.02,2.259892,0.56,0.368
9,ABC,109.810,455,-6.75,0.38,-31.3100,0.08,0.1159,0.02,7.646935,0.6,3.661924,0.78,0.372


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [177]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine = 'xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name = 'Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [178]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [179]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile', percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template],
}

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    #writer.sheets['Value Strategy'].write(f'{column}l', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [180]:
writer.save()